In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
import sys
import random
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [7]:
# # Load Hydra config in notebooks
# # https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
# import os
# from hydra import initialize_config_dir, compose
# from hydra.core.config_store import ConfigStore
# from omegaconf import OmegaConf
# from config import Grid_Environment

## cs = ConfigStore.instance()
## cs.store(name="env", node=Grid_Environment)

# abs_config_dir=os.path.abspath("config/")
# with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
#     config_env = compose(config_name="env.yaml", overrides=[])
#     # print(OmegaConf.to_yaml(env_config))
#     print(config_env)

In [16]:
# environment
import config
train = config.env.BufferSpec(
    path="~/scratch/datasets/grid/corners_d2l3_r0.csv",
    seed=167,
    n=1000,
)
test = config.env.BufferSpec(
    path="~/scratch/datasets/grid/corners_d2l3_r0.csv",
    seed=167,
    n=1000,
)
buffer = config.env.Buffer(train=train, test=test)
config_env = config.env.Grid_Env(
    buffer, 
    min_step_len=1,
    max_step_len=1,
    cell_min=-1,
    cell_max=1
)

from environment.grid_env import GridEnv
env = GridEnv(**config_env.__dict__)